<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Evaluating Classifiers: The Confusion Matrix and AUC-ROC

_Authors: Kiefer Katovich (SF), Joseph Nelson (DC)_

_Modified for DSI-EAST by Justin Pounders(ATL)_

---

### Learning Objectives
- Define true positives, false positives, true negatives, and false negatives
- Create and interpret a confusion matrix
- Calculate the common classification evaluation metrics
- Describe how the threshold affects predictions in a classification model
- Define and plot the receiver operating characteristic (ROC) curve

### Lesson Guide
- [Introduction: evaluating classifiers](#intro)
- [The baseline accuracy](#baseline)
- [Build a KNN model to predict spam](#knn)
- [Predicting labels vs. predicting probabilities](#labels-vs-probs)
- [The confusion matrix](#confusion-matrix)
- [Review: Type I error and p-values](#type1-pvalues)
- [Type II error and power](#type2-power)
- [Fundamental classifier metrics](#metrics)
    - [Accuracy](#accuracy)
    - [Sensitivity / Recall / True Positive Rate (TPR)](#sensitivity)
    - [False Positive Rate (FPR)](#fpr)
    - [Specificity / True Negative Rate (TNR)](#specificity)
    - [Precision / Positive Predictive Value](#precision)
- [The F1-score and sklearn's `classification_report`](#f1-score)
- [Changing the threshold for prediction](#threshold)
    - [Load the UCI breast cancer data](#uci)
    - [Evaluate prediction on a test set](#testing)
- [The ROC curve](#roc-curve)
- [Reference table of common classification metric terms and definitions](#table)
- [Additional resources](#resources)


In [1]:
import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

<a id='intro'></a>

## Introduction: evaluating classifiers

---

The evaluation of classifier models is not as straightforward as regression models. In this lesson we will cover the fundamentals of evaluating classifiers.

To ground the theory in a real world example, we will use a spam dataset to build and evaluate classifiers on. The spam dataset is 1001 columns wide, containing an `is_spam` binary marker for whether a given email was spam or not, and then 1000 columns, each correpsonding to a word that could have appeared in the email (also marked with 0 or 1).

**Load the spam data below and print out the header.**

In [2]:
spam = pd.read_csv('./datasets/spam_words_wide.csv')

In [3]:
spam.head()

,is_spam,getzed,86021,babies,sunoco,ultimately,thk,voted,spatula,fiend,...,itna,borin,thoughts,iccha,videochat,freefone,pist,reformat,strict,69698
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(spam.shape, spam.is_spam.mean())

(5572, 1001) 0.13406317300789664


<a id='baseline'></a>

## The baseline accuracy

---

The importance of calculating your baseline accuracy when building classifiers cannot be overstated. It is critical to know the baseline when you are evaluating a classifier using accuracy.

> **Baseline Accuracy**: The accuracy that can be achieved by a model by simply guessing the majority class for every observation.

As human beings we are inclined to think that "50% accuracy" is equivalent to guessing by chance. In fact, a 50% accuracy only equates to guessing by chance in a very specific context: when we have equal proportion of positive and negatvie (1 and 0) target class labels in our dataset, or in the multi-class case when the majority class makes up 50% of the labels.

> **`baseline_accuracy = majority_class_N / total_N`**

In a binary class problem the reality is that your dataset is more likely to be unbalanced, and the more unbalanced it is the higher the baseline accuracy becomes. This is important to remember because if 99% of your observations are of one class, predicting 99% of them correctly with a model is performing at chance.

#### Calculate the baseline accuracy for the spam dataset

In [5]:
baseline_acc = 1 - spam['is_spam'].mean()

In [6]:
baseline_acc # class imbalance

0.8659368269921034

In [7]:
baseline_acc

0.8659368269921034

<a id='knn'></a>

## Set up a KNN model to predict spam

---

We will use just the first 250 columns for the sake of speed in fitting and prediction. Even with a fourth of the predictors the cross-validation can be noticeably slow.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

y = spam.is_spam.values
X = spam.iloc[:, 1:]

In [9]:
knn = KNeighborsClassifier()

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
accs = cross_val_score(knn, X, y, cv=10)
print(accs)
print(np.mean(accs))

[ 0.91935484  0.93369176  0.9265233   0.9265233   0.93010753  0.92100539
  0.92459605  0.9352518   0.94064748  0.9442446 ]
0.9301946038


In [12]:
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [13]:
from sklearn.neighbors import KNeighborsClassifier

y = spam['is_spam'].values
x = spam.iloc[:, 1:251]

knn  = KNeighborsClassifier()

#### Cross-validate the accuracy of the model

Use 10 folds. How does the mean performace across folds compare to the baseline accuracy?

In [14]:
from sklearn.model_selection import cross_val_score

accs = cross_val_score(knn, x, y, cv=10)
print(accs)
print(np.mean(accs))

[ 0.89784946  0.88351254  0.88888889  0.87634409  0.89605735  0.88330341
  0.89228007  0.89208633  0.90827338  0.91366906]
0.893226458967


<a id='labels-vs-probs'></a>

## Predicting labels vs. predicting probabilities

---

Sklearn classification models come with two distinct prediction functions:
1. `.predict()`: predicts the labels (classes) of observations 
2. `.predict_proba()` predict the *probability of membership to each class*.

The `.predict()` function will return the predicted labels for a design matrix as a vector of integer labels. 

In contrast, the `.predict_proba()` function will return the probabilities as a matrix, where the columns are ordered in increasing order of the class labels (eg. the first column is probabilities for class 0, the second column is probabilities for class 1).

**Fit the KNN model and print out the predicted labels and predicted probabilities for a few points.**

In [15]:
knn.fit(X,y)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [16]:
knn.predict(X.iloc[0:10, :])

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int64)

In [17]:
print(y[0:10])

[0 0 1 0 0 1 0 0 1 1]


In [18]:
knn.predict_proba(X.iloc[0:10, :])

array([[ 1. ,  0. ],
       [ 0.8,  0.2],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 0.8,  0.2],
       [ 0.6,  0.4],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 0.2,  0.8],
       [ 0. ,  1. ]])

<a id='confusion-matrix'></a>

## The confusion matrix

---

The confusion matrix is a table representing the performance of your model to classify labels correctly.

**A confusion matrix for a binary classification task:**

|   |Predicted Negative | Predicted Positive |   
|---|---|---|
|**Actual Negative**  | True Negative (TN)  | False Positive (FP)  |
|**Actual Positive** | False Negative (FN)  | True Positive (TP)  |

In a binary classifier, the "true" class is typically labeled with 1 and the "false" class is labeled with 0. 

> **True Positive**: A positive class observation (1) is correctly classified as positive by the model.

> **False Positive**: A negative class observation (0) is incorrectly classified as positive.

> **True Negative**: A negative class observation is correctly classified as negative.

> **False Negative**: A positive class observation is incorrectly classified as negative.

Columns of the confusion matrix sum to the predictions by class. Rows of the matrix sum to the actual values within each class. You may encounter confusion matrices where the actual is in columns and the predicted is in the rows: the meaning is the same but the table will be reoriented.

> **Note:** Remembering what the cells in the confusion matrix represents can be a little tricky. The first word (True or False) indicates whether or not the model was correct. The second word (Positive or Negative) indicates the *model's guess* (not the actual label!).

#### Calculate the confusion matrix metrics for your model below.

In [20]:
predicted = knn.predict(X)

In [23]:
tp = np.sum((y == 1)) & (predicted == 1)

In [24]:
tn = np.sum((y == 0) & (predicted == 0))
fn = np.sum((y == 1) & (predicted == 0))
fp = np.sum((y == 0) & (predicted == 1))

In [27]:
print(tn)
print(fn)
print(tp)
print(fp)

4810
314
[0 0 0 ..., 0 0 0]
15


#### Verify this is the same as the numbers you get from sklearn's `metrics.confusion_matrix`

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(y, predicted) # orde of numbers follows the gird

array([[4810,   15],
       [ 314,  433]], dtype=int64)

<a id='type1-pvalues'></a>

## Review: Type I error and p-values

---

In the context of hypothesis testing false positives and false negatives are referred to as Type I and Type II error, respectively. 

Type I error is the incorrect rejection of the null hypothesis when in fact the null hypothesis is true. This is equivalent to the false positive rate in classification: the rate of a model labeling an observation as "true" when in fact it is "false". 

Type I error directly corresponds to p-values: **the p-value is the probability of incorrectly rejecting the null hypothesis.**

<a id='metrics'></a>

## The fundamental classifier metrics

---

All metrics we use to evaluate classifiers are tied to the content of the confusion matrix. Remember that for the binary classification case we have four cells in the confusion matrix:

- **`tp`**: true positives (classifier correct; classifier guessed 1)
- **`fp`**: fasle positives (classifier incorrect; classifier guessed 1)
- **`tn`**: true negative (classifier correct; classifier guessed 0)
- **`fn`**: false negative (classifier incorrect; classifier guessed 0)

Below are the fundamental metrics that data scientists use to evaluate the performance of their classifier model.

---

<a id='accuracy'></a>
### Accuracy

With the total population as:

> **`total_population = tp + fp + tn + fn`**

The accuracy can be calculated as:

> **`accuracy = (tp + tn) / total_population`**

Which is just the proportion of correct guesses, regardless of class. The `.score()` function attached to sklearn classification model objects defaults to returning the accuracy of the model's predictions given an `X` and `y`.

The inverse of the accuracy is known as the **misclassification rate**, which is calculated:

> **`misclassification_rate = (fp + fn) / total_population`**

**Calculate the accuracy using the confusion matrix cells.**
- Validate that it is the same as `metrics.accuracy_score`

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y, predicted)

0.94095477386934678

---
<a id='sensitivity'></a>
### Sensitivity / Recall / True Positive Rate

The true positive rate is the percent of times that when the label was in fact 1 the model predicted 1. This is alternatively known as the **Sensitivity** or **Recall**. 

This is calculated as:

> **`recall = tp / (tp + fn)`**

**Calculate the recall with the confusion matrix cells.**
- Validate that this is the same as `metrics.recall_score`

In [29]:
from sklearn.metrics import recall_score

print(recall_score(y, predicted))
print(tp / (tp + fn))

0.579651941098
[ 0.  0.  0. ...,  0.  0.  0.]


---
<a id='fpr'></a>
### False Positive Rate

Alternatively, the false positive rate measures the percent of times the model predicts a 1 when the target class is actually a 0. 

> **`fpr = fp / (tn + fp)`**

**Calculate the FPR using the confusion matrix cells.**

---
<a id='specificity'></a>
### Specificity / True Negative Rate

The true negative rate measures the percent of times the classifier predicted the class was 0 out the number of actual times the class was 0. It is the sister metric to Sensitivity, which measures the same thing but for positives.

> **`specificity = tn / (tn + fp)`**

**Calculate the specificity using the confusion matrix cells.**

---
<a id='precision'></a>
### Precision / Positive Predictive Value

The precision measures the percent of times that the classifier was correct when it was predicting the true (1) class.

> **`precision = tp / (tp + fp)`**

The idea of the classifier being _precise_ is subtly different than it being _accurate_. Precision is a measure of correctness only for its positive class predictions, whereas accuracy is a measure of correctness for all guesses.

**Calculate the precision using the confusion matrix cells.**
- Validate that this is the same as `metrics.precision_score`

In [30]:
from sklearn.metrics import precision_score

precision_score(y, predicted)

0.9665178571428571

<a id='f1-score'></a>

## F1-score and the `classification_report`

---

Sklearn's `metrics.classification_report` helps diagnose the effectiveness of your classifier. The report focuses on the precision, recall, and a metric known as the f1-score.

The f1-score is the [harmonic mean](https://en.wikipedia.org/wiki/Harmonic_mean) of the precision and recall metrics. Blending the two is useful: precision measures how effectively the classifier performs when it is predicting a 1, whereas recall measures how many of the total 1 classes out of all the 1-labeled observations were predicted correctly. 

### $$ F_1 = 2 \cdot \frac{1}{\tfrac{1}{\mathrm{recall}} + \tfrac{1}{\mathrm{precision}}} = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}}$$

By combining the two we have a measure of the classifiers ability to find the positive labeled observations as well as how permissive it is of identification errors on those labels.

**A brief overview/guide:**
- `Precision = True Positives / (True Positives + False Positives)`
    - A precision score of 1 indicates that the classifier never mistakenly added observations from another class. A precision score of 0 would mean that the classifier misclassified every instance of the current class.
- `Recall = True Positives / (True Positives + False Negatives)`
    - A recall score of 1 indicates that the classifier correctly predicted (found) all observations of the current class (by implication, no false negatives, or misclassifications of the current class). A recall score of 0 alternatively means that the classifier missed all observations of the current class.
- `F1-Score = 2 * (Precision * Recall) / (Precision + Recall)`
    - The f1-score's best value is 1 and worst value is 0, like the precision and recall scores. It is a useful metric for taking into account both measures at once.
- `Support` is simply the number of observations of the labelled class.

You can print out the report of these three metrics on both of the classes (or more if you have a multi-class problem) using the `classification_report` function.

In [31]:
from sklearn.metrics import classification_report

In [32]:
print(classification_report(y, predicted))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97      4825
          1       0.97      0.58      0.72       747

avg / total       0.94      0.94      0.93      5572



<a id='threshold'></a>

## Changing the threshold for prediction

---

The prediction of the classifier defaults to guessing the class that has the highest predicted probability. This neccessarily leads to the highest possible accuracy (**only a guarantee for the training data!**). 

However, it could be the case that maximizing the accuracy is not, in fact, our ultimate goal. Consider the following scenario:

> **Cancer detection:** You have developed a classifier to detect, based on some medical measurements, whether or not a person has a cancerous tumor or not. Your classifier gets a 96% accuracy compared to a 60% baseline accuracy.

Your classifier is performing well, but what might be wrong with just maximizing the accuracy in this case? Think back to the confusion matrix and your goal (to treat cancer patients before its too late).

--- 
<a id='uci-data'></a>
### Load the UCI breast cancer data

Below we will load the medical data on breast cancer detection.

In [ ]:
column_names = ['id',
                'clump_thickness',
                'cell_size_uniformity',
                'cell_shape_uniformity',
                'marginal_adhesion',
                'single_epithelial_size',
                'bare_nuclei',
                'bland_chromatin',
                'normal_nucleoli',
                'mitoses',
                'class']

bcw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', 
                  names=column_names)

bcw['bare_nuclei'] = bcw.bare_nuclei.map(lambda x: int(x) if not x == '?' else np.nan)
bcw.dropna(inplace=True)

y = bcw['class'].map(lambda x: 1 if x == 4 else 0)
X = bcw.iloc[:, 1:-1]

**Split the data into 66% training and 33% testing. Fit a KNN classifier with `n_neighbors=25` on the training data.**

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33)
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
           weights='uniform')

**Calculate the accuracy on the test set and compare to baseline.**

In [34]:
baseline = 1 - np.mean(y_test)
print(knn.score(X_test, y_test), baseline)

0.883632408918 0.859706362153


**Calculate the predicted labels and predicted probabilities on the test set.**

In [35]:
y_pred = knn.predict(X_test)
y_pp = knn.predict_proba(X_test)
y_pp

array([[ 0.92,  0.08],
       [ 0.6 ,  0.4 ],
       [ 0.96,  0.04],
       ..., 
       [ 0.96,  0.04],
       [ 1.  ,  0.  ],
       [ 1.  ,  0.  ]])

**Create the confusion matrix for your classfier's performance on the test set.**

In [ ]:
conmat = confusion_matrix(y_test, y_pred)
confusion = pd.DataFrame(conmat, 
                         index=['is_healthy', 'is_cancer'],
                         columns=['predicted_healthy', 'predicted_cancer'])
confusion

<a id='change-threshold'></a>

### Lower the threshold for predicting cancer

Right now the classifier is choosing to label cancer vs. healthy based on the 0.5 predicted probability threshold. 

Say our goal was to have 0 false negatives: in other words, in no case do we want to predict the person is healthy when in fact they have cancer!

1. Create a dataframe of the predicted probabilities (class 0 and class 1 probabilities).
2. Create a new column with predicted labels where the threshold for labeling cancer/1 is 10% rather than 50%
    - In other words, the predicted probability of class 1 only needs to be greater than 0.10 for the label to be 1.
3. Recreate the confusion matrix with the predictions using the new threshold.

In [ ]:
y_pp = pd.DataFrame(knn.predict_proba(X_test),
                    columns=['class_0_pp','class_1_pp'])
y_pp.head()

In [ ]:
y_pp['pred_class_thresh10'] = [1 if x>= 0.1 else 0 for x in y_pp.class_1_pp.values]

In [ ]:
y_pp.head()

In [ ]:
conmat = confusion_matrix(y_test, y_pp.pred_class_thresh10)
confusion = pd.DataFrame(conmat,
                         index=['is_healthy', 'is_cancer'],
                         columns=['predicted_healthy', 'predicted_cancer'])
confusion

<a id='roc-curve'></a>

## The Receiver operating characteristic (ROC) curve

---

The ROC curve is a popular visual of the performance of a classifier. It has a few attractive properties:
- It compares the true positive rate to the false positive rate **as the threshold for predicting 1 changes**.
- When the area under the curve is 0.50, this is equivalent to the baseline (chance) prediction.
- When the area under the curve is 1.00, this is equivalent to perfect prediction.

The area under the ROC curve is inherently related to the accuracy, but the AUC-ROC is preferred because it is automatically adjusted to the baseline and gives a robust picture of how the classifier performs at different threshold choices. 

**Note:**
- As the class assignment threshold increases for the positive class (has cancer), the false positive rate and true positive rate necessarily increase.
- For a classifier performing at chance, this is the diagonal dotted line: an equal chance of false positives and true positives.
- The greater the area under the curve, the higher the ratio of true positives to false positives as the threshold becomes more lenient.
- The greater the area under the curve, the higher the quality of the classification model. 
- Most classification problems will never get close to a full 1.0 area under the curve: the wisconsin breast cancer data is unique in that the signal is extremely strong. If you see this, it usually indicates that something is wrong with your procedure (or, if you are predicting on the training set, that your model is overfitting.)

**Below is code to plot the ROC curve for our cancer data.**

In [ ]:
from sklearn.metrics import roc_curve, auc # auc= a read under curve

In [ ]:
fpr, tpr, thresh= roc_curve(y_test, y_pp.class_1_pp)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=[8,8])
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc, linewidth=4)
plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic for cancer detection', fontsize=18)
plt.legend(loc="lower right")
plt.show()

In [ ]:
np.array([i for i in zip(tpr, fpr, thresh)])

---

## Reference table of common classification metric terms and definitions

<br><br>

|  TERM | DESCRIPTION  |
|---|---|
|**TRUE POSITIVES** | The number of "true" classes correctly predicted to be true by the model. <br><br> `TP = Sum of observations predicted to be 1 that are actually 1`<br><br>The true class in a binary classifier is labeled with 1.|
|**TRUE NEGATIVES** | The number of "false" classes correctly predicted to be false by the model. <br><br> `TP = Sum of observations predicted to be 0 that are actually 0`<br><br>The false class in a binary classifier is labeled with 0.|
|**FALSE POSITIVES** | The number of "false" classes incorrectly predicted to be true by the model. This is the measure of **Type I error**.<br><br> `TP = Sum of observations predicted to be 1 that are actually 0`<br><br>Remember that the "true" and "false" refer to the veracity of your guess, and the "positive" and "negative" component refer to the guessed label.|
|**FALSE NEGATIVES** | The number of "true" classes incorrectly predicted to be false by the model. This is the measure of **Type II error.**<br><br> `TP = Sum of observations predicted to be 0 that are actually 1`<br><br>|
|**TOTAL POPULATION** | In the context of the confusion matrix, the sum of the cells. <br><br> `total population = tp + tn + fp + fn`<br><br>|
|**SUPPORT** | The marginal sum of rows in the confusion matrix, or in other words the total number of observations belonging to a class regardless of prediction. <br><br>|
|**ACCURACY** | The number of correct predictions by the model out of the total number of observations. <br><br> `accuracy = (tp + tn) / total_population`<br><br>|
|**PRECISION** | The ability of the classifier to avoid labeling a class as a member of another class. <br><br> `Precision = True Positives / (True Positives + False Positives)`<br><br>_A precision score of 1 indicates that the classifier never mistakenly classified the current class as another class.  precision score of 0 would mean that the classifier misclassified every instance of the current class_ |
|**RECALL/SENSITIVITY**    | The ability of the classifier to correctly identify the current class. <br><br>`Recall = True Positives / (True Positives + False Negatives)`<br><br>A recall of 1 indicates that the classifier correctly predicted all observations of the class.  0 means the classifier predicted all observations of the current class incorrectly.|
|**SPECIFICITY** | Percent of times the classifier predicted 0 out of all the times the class was 0.<br><br> `specificity = tn / (tn + fp)`<br><br>|
|**FALSE POSITIVE RATE** | Percent of times model predicts 1 when the class is 0.<br><br> `fpr = fp / (tn + fp)`<br><br>|
|**F1-SCORE** | The harmonic mean of the precision and recall. The harmonic mean is used here rather than the more conventional arithmetic mean because the harmonic mean is more appropriate for averaging rates. <br><br>`F1-Score = 2 * (Precision * Recall) / (Precision + Recall)` <br><br>_The f1-score's best value is 1 and worst value is 0, like the precision and recall scores. It is a useful metric for taking into account both measures at once._ |

<a id='resources'></a>

## Additional resources

---

- An introduction to [Confusion Matrix terminology](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/)
- A deeper [Introduction to ROC](http://people.inf.elte.hu/kiss/13dwhdm/roc.pdf)
- Receiver Operation Characteristic curves, [university resource](http://ebp.uga.edu/courses/Chapter%204%20-%20Diagnosis%20I/8%20-%20ROC%20curves.html)
- Interactive [playing with ROC curves](http://www.navan.name/roc/)
- Data School's video and transcript on [ROC/AUC](http://www.dataschool.io/roc-curves-and-auc-explained/)
- Watch Rahul Patwari's [video](https://www.youtube.com/watch?v=21Igj5Pr6u4) on ROC curves